In [ ]:
""" import settings """
%load_ext autoreload
%autoreload 2

import numpy as np
import sys, os, csv

from utils_invitro_global import *

from utils_invitro_misc import parse_Hf_filename

In [ ]:
from matplotlib import pyplot as plt
from matplotlib_settings import set_plot_settings, reset_plot_settings

set_plot_settings()

In [ ]:
"""
Transfer function was characterized by injecting sinusoid into the electrolyte
and taking either 16x16 or 32x32 recording that covers a quarter of the array (centered at array center)
coverage avoided including pixels at the edge of the array because of packaging limitations

each file contains pixel gains that represent a particular combination of :
chip config (PGA gain, HPF cut-off, 256 vs 1024) and injected sinusoid frequency
"""

In [ ]:
""" fetch recording data file names """
data_dir = f'{REC_DATA_DIR}/sinusoid_sweep'
file_list = [fn for fn in os.listdir(data_dir) if fn.endswith('.npy')]

Part 1. Plot Transfer Function vs. PGA configuration

Choose HPF and Tetrode Mode Configs

In [ ]:
""" choose which configs to plot: HPF, 256 vs 1024 """
VBIAS_PR       = 6 # choose from 5, 6, 7. 5: lower cut-off. 7: higher-cut-off
EN_STATIC_ELEC = 0 # choose from 0, 1. 0: 1024, 1: 256

In [ ]:
""" Fetch pixel gain data for the given HPF, EN_STATIC_ELEC """
# gain_mu, gain_sigma, valid_npt, freq: gain mean, gain SD, # of non-saturated channels, input frequency
gain_mu0, gain_sigma0, valid_npt0, freqs0 = [], [], [], [] # vga_gain = 0
gain_mu1, gain_sigma1, valid_npt1, freqs1 = [], [], [], [] # vga_gain = 1
gain_mu2, gain_sigma2, valid_npt2, freqs2 = [], [], [], [] # vga_gain = 2
gain_mu3, gain_sigma3, valid_npt3, freqs3 = [], [], [], [] # vga_gain = 3

for fn in file_list:
    vga_gain, vbias_pr, en_static_elec, input_freq = parse_Hf_filename(fn)

    if not (vbias_pr == VBIAS_PR and en_static_elec == EN_STATIC_ELEC):
        continue

    array_gain = np.load(f'{data_dir}/{fn}')
    array_gain = 20*np.log10(array_gain)

    mu = np.nanmean(array_gain)
    sigma = np.nanstd(array_gain)
    valid_npt = array_gain.size - np.isnan(array_gain).sum()

    if vga_gain == 0:
        gain_mu0.append(mu)
        gain_sigma0.append(sigma)
        valid_npt0.append(valid_npt)
        freqs0.append(input_freq)
    if vga_gain == 1:
        gain_mu1.append(mu)
        gain_sigma1.append(sigma)
        valid_npt1.append(valid_npt)
        freqs1.append(input_freq)
    if vga_gain == 2:
        gain_mu2.append(mu)
        gain_sigma2.append(sigma)
        valid_npt2.append(valid_npt)
        freqs2.append(input_freq)
    if vga_gain == 3:
        gain_mu3.append(mu)
        gain_sigma3.append(sigma)
        valid_npt3.append(valid_npt)
        freqs3.append(input_freq)
        
# convert to numpy arrays
gain_mu0, gain_sigma0, valid_npt0, freqs0  = np.array(gain_mu0), np.array(gain_sigma0), np.array(valid_npt0), np.array(freqs0) 
gain_mu1, gain_sigma1, valid_npt1, freqs1  = np.array(gain_mu1), np.array(gain_sigma1), np.array(valid_npt1), np.array(freqs1) 
gain_mu2, gain_sigma2, valid_npt2, freqs2  = np.array(gain_mu2), np.array(gain_sigma2), np.array(valid_npt2), np.array(freqs2) 
gain_mu3, gain_sigma3, valid_npt3, freqs3  = np.array(gain_mu3), np.array(gain_sigma3), np.array(valid_npt3), np.array(freqs3) 

In [ ]:
""" sort by frequency """
assert np.array_equal(freqs0, freqs1)
assert np.array_equal(freqs0, freqs2)
assert np.array_equal(freqs0, freqs3) 

sorted_idxs = np.argsort(freqs0)

freqs       = freqs0     [sorted_idxs]

gain_mu0    = gain_mu0   [sorted_idxs]
gain_sigma0 = gain_sigma0[sorted_idxs]
valid_npt0  = valid_npt0 [sorted_idxs]

gain_mu1    = gain_mu1   [sorted_idxs]
gain_sigma1 = gain_sigma1[sorted_idxs]
valid_npt1  = valid_npt1 [sorted_idxs]

gain_mu2    = gain_mu2   [sorted_idxs]
gain_sigma2 = gain_sigma2[sorted_idxs]
valid_npt2  = valid_npt2 [sorted_idxs]

gain_mu3    = gain_mu3   [sorted_idxs]
gain_sigma3 = gain_sigma3[sorted_idxs]
valid_npt3  = valid_npt3 [sorted_idxs]

In [ ]:
""" plot """
fig, ax = plt.subplots(1, 1, figsize=(4, 3))

ax.errorbar(freqs, gain_mu0, yerr=gain_sigma0/np.sqrt(valid_npt0), fmt='o-', label='mu', capsize=6)
ax.errorbar(freqs, gain_mu1, yerr=gain_sigma1/np.sqrt(valid_npt1), fmt='o-', label='mu', capsize=6)
ax.errorbar(freqs, gain_mu2, yerr=gain_sigma2/np.sqrt(valid_npt2), fmt='o-', label='mu', capsize=6)
ax.errorbar(freqs, gain_mu3, yerr=gain_sigma3/np.sqrt(valid_npt3), fmt='o-', label='mu', capsize=6)
ax.set_xscale('log')

ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Gain (dB)')
ax.grid(True)
ax.set_xticks([1, 100, 1e4])
if en_static_elec == 1:
    ax.set_title(f'Non-Tetrode. Vbias_PR={VBIAS_PR}\n')
else:
    ax.set_title(f'Tetrode. Vbias_PR={VBIAS_PR}\n')

legend = ax.legend(['0', '1', '2', '3'], title='PGA Gain',
                   title_fontsize=14, fontsize=14, loc=(1.05, 0))

fb_idx = np.where(freqs == 1e3)[0][0]
print(f'Flat Band Gains (mean): {gain_mu0[fb_idx]:.2f}, {gain_mu1[fb_idx]:.2f}, {gain_mu2[fb_idx]:.2f}, {gain_mu3[fb_idx]:.2f} dB') 
print(f'Flat Band Gains (SD): {gain_sigma0[fb_idx]:.2f}, {gain_sigma1[fb_idx]:.2f}, {gain_sigma2[fb_idx]:.2f}, {gain_sigma3[fb_idx]:.2f} dB') 
print(f'Flat Band Gains (N): {valid_npt0[fb_idx]}, {valid_npt1[fb_idx]}, {valid_npt2[fb_idx]}, {valid_npt3[fb_idx]}') 

Part 2. Plot Transfer Function vs. HPF configuration

Choose PGA Gain and Tetrode Mode Configs

In [ ]:
""" choose which configs to plot: VGA_GAIN, 256 vs 1024 """
VGA_GAIN       = 2 # choose from 0, 1, 2, 3.  0: lowest gain. 3: highest gain
EN_STATIC_ELEC = 1 # choose from 0, 1. 0: 1024, 1: 256

In [ ]:
""" Fetch pixel gain data for the given VGA_GAIN, EN_STATIC_ELEC """
# gain_mu, gain_sigma, valid_npt, freq: gain mean, gain SD, # of non-saturated channels, input frequency
gain_mu5, gain_sigma5, valid_npt5, freqs5 = [], [], [], [] # vbias_pr = 5. vbias_pr; voltage bias for pseudo-resistor
gain_mu6, gain_sigma6, valid_npt6, freqs6 = [], [], [], [] # vbias_pr = 6
gain_mu7, gain_sigma7, valid_npt7, freqs7 = [], [], [], [] # vbias_pr = 7

for fn in file_list:
    vga_gain, vbias_pr, en_static_elec, input_freq = parse_Hf_filename(fn)

    if not (vga_gain == VGA_GAIN and en_static_elec == EN_STATIC_ELEC):
        continue
      
    array_gain = np.load(f'{data_dir}/{fn}')
    array_gain = 20*np.log10(array_gain)

    mu = np.nanmean(array_gain)
    sigma = np.nanstd(array_gain)
    valid_npt = array_gain.size - np.isnan(array_gain).sum()

    if vbias_pr == 5:
        gain_mu5.append(mu)
        gain_sigma5.append(sigma)
        valid_npt5.append(valid_npt)
        freqs5.append(input_freq)
    if vbias_pr == 6:
        gain_mu6.append(mu)
        gain_sigma6.append(sigma)
        valid_npt6.append(valid_npt)
        freqs6.append(input_freq)
    if vbias_pr == 7:
        gain_mu7.append(mu)
        gain_sigma7.append(sigma)
        valid_npt7.append(valid_npt)
        freqs7.append(input_freq)
              
# convert to numpy arrays
gain_mu5, gain_sigma5, valid_npt5, freqs5  = np.array(gain_mu5), np.array(gain_sigma5), np.array(valid_npt5), np.array(freqs5) 
gain_mu6, gain_sigma6, valid_npt6, freqs6  = np.array(gain_mu6), np.array(gain_sigma6), np.array(valid_npt6), np.array(freqs6) 
gain_mu7, gain_sigma7, valid_npt7, freqs7  = np.array(gain_mu7), np.array(gain_sigma7), np.array(valid_npt7), np.array(freqs7) 

In [ ]:
""" sort by frequency """
assert np.array_equal(freqs5, freqs6)
assert np.array_equal(freqs5, freqs7)

sorted_idxs = np.argsort(freqs5)

freqs       = freqs5     [sorted_idxs]

gain_mu5    = gain_mu5   [sorted_idxs]
gain_sigma5 = gain_sigma5[sorted_idxs]
valid_npt5  = valid_npt5 [sorted_idxs]

gain_mu6    = gain_mu6   [sorted_idxs]
gain_sigma6 = gain_sigma6[sorted_idxs]
valid_npt6  = valid_npt6 [sorted_idxs]

gain_mu7    = gain_mu7   [sorted_idxs]
gain_sigma7 = gain_sigma7[sorted_idxs]
valid_npt7  = valid_npt7 [sorted_idxs]

In [ ]:
from scipy.interpolate import interp1d

def print_3dB_HPC(freq, gain_mu, gain_sigma, fb_gain, valid_npt):
    """
    given transfer function measurements taken in a coarse frequency range,
    1. interpolates the data to a finer resolution range
    2. interpolates the 3-dB HPF cut-off and prints out the result

    freq: measured frequencies
    gain_mu: pixel gain, average. across frequencies
    gain_sigma: pixel gain, SD. across frequencies
    fb_gain: pixel gain, average @ flat band frequency 
    valid_npt: number of samples at each data point
    """
    f_mu = interp1d(freq, gain_mu)
    f_sigma = interp1d(freq, gain_sigma)
    f_npt = interp1d(freq, valid_npt)

    finer_freq = np.logspace(0, 3, 300)
    finer_mu = f_mu(finer_freq)
    # interpolating SD is okay because it is behaving monotonically in
    # the frequency range of our interest
    finer_sigma = f_sigma(finer_freq) 
    finer_npt = f_npt(finer_freq)

    idx = np.where(finer_mu <= fb_gain - 3)[0][-1]
    print(f'3dB HPC: {finer_freq[idx]:.2f} ± {finer_sigma[idx]:.2f} Hz, n = {int(finer_npt[idx])}')

# confirm monotonicity of SD
# fig, ax = plt.subplots(figsize=(4, 3))
# ax.plot(freqs, gain_sigma5)
# ax.plot(freqs, gain_sigma6)
# ax.plot(freqs, gain_sigma7)
# ax.set_xscale('log')

In [ ]:
""" print 3-dB cut-offs """
fb_idx = np.where(freqs == 1e3)[0][0]
fb_gain = np.mean((gain_mu5[fb_idx], gain_mu6[fb_idx], gain_mu7[fb_idx]))

print_3dB_HPC(freqs, gain_mu5, gain_sigma5, fb_gain, valid_npt5)
print_3dB_HPC(freqs, gain_mu6, gain_sigma6, fb_gain, valid_npt6)
print_3dB_HPC(freqs, gain_mu7, gain_sigma7, fb_gain, valid_npt7)

In [ ]:
""" plot """
fig, ax = plt.subplots(1, 1, figsize=(4, 3))

ax.errorbar(freqs, gain_mu7, yerr=gain_sigma7/np.sqrt(valid_npt7), fmt='o-', label='mu', capsize=6)
ax.errorbar(freqs, gain_mu6, yerr=gain_sigma6/np.sqrt(valid_npt6), fmt='o-', label='mu', capsize=6)
ax.errorbar(freqs, gain_mu5, yerr=gain_sigma5/np.sqrt(valid_npt5), fmt='o-', label='mu', capsize=6)
ax.set_xscale('log')

ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Gain (dB)')
ax.set_xticks([1, 100, 1e4])
ax.set_yticks([20, 40, 60])
ax.grid(True)